Before running the code, several dependant package need to be installed, skip if it already exists.

In [1]:
pip install urllib3

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pdfminer.six

     |████████████████████████████████| 5.6MB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install python-Levenshtein

Firstly, we make connection with database with pymysql api.

In [ ]:
import pymysql
#https://cloud.tencent.com/developer/article/1395339
import urllib
import pdfminer
import copy
from io import StringIO
from io import BytesIO
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import *
from pdfminer.converter import PDFPageAggregator
from urllib.request import Request
from urllib.request import urlopen
import os.path
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef,RDFS
from rdflib.namespace import DC, FOAF
from owlrl import DeductiveClosure, RDFS_Semantics
import pprint
import json
from pathlib import Path
#input basic info about database
class MySQL_Processing():
    '''
    This class is used to connect with the mysql database, download the paper and convert to the txt file
    The result return resourceId(string),authorList(List),authorPaper,authorCitation and authorID for further
    processing
    '''
    def __init__(self):
        self.NUM=3
        self.conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')

    def connection(self):
        '''
        Make connection with given mysql database and excute the given query
        Returns:
            A list containing of the mysql query result
        '''
        query_result=[]
        #cursor is the current point we focus on, which could execute sql command.
        cursor=self.conn.cursor()
        #Here rw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
        cursor.execute("SELECT * FROM krw.training_paper_view group by id desc limit %s;",self.NUM)
        for r in cursor:
            query_result.append(list(r))
        cursor.close()
        return query_result
    
    def pre_processing(self,index,query_result):
        '''
        Handle each sublist in the query list by index
        Args:
            index: int as query_result index
            query_result: List genearted by the connection function
        Returns:
            resourceId: string represents each paper's unique ID, e.g. P12345
            url: string url where pdf file can be obtained
        '''
        resourceId=query_result[index][2]
        authorNum=query_result[index][3]
        url=query_result[index][7]
        webList=["acmweb","aclweb"]
        for web in webList:
            if web in url:
                url=url+".pdf"
        print(resourceId,url)
        return resourceId,url
        
    def query(self,index, resourceId):
        '''
        Extract authors' related information in the query list by index and resouceId
        Args:
            index: int as query_result index
            resourceId: string represents each paper's unique ID, e.g. P12345
        Returns:
            authorList: A list containing strings of all the names of authors
            authorPaper: A dictionary containing integers of all the paper count of authors
            authorCitation: : A dictionary containing integers of all the citation counts of authors
            authorID: A dictionary containing strings of all the strings ID of authors. e.g. A12345
        '''
        temp=[]
        authorList=[]
        #Here krw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
        #conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
        cursor=self.conn.cursor()
        #Here krw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
        cursor.execute("SELECT * FROM krw.training_author_view where paper_id=(%s) group by name;",resourceId)
        cursor.close()
        authorPaper={}
        authorCitation={}
        authorID={}
        for r in cursor:
            temp.append(list(r))
        #print(temp)
        for line in temp:
            authorList.append(line[1])
            authorPaper[line[1]]=[line[3]]
            authorCitation[line[1]]=[line[4]]
            authorID[line[1]]=[line[0]]
        #print(authorList)
        return authorList,authorPaper,authorCitation,authorID

    def onlinePdfToTxt(self,dataIo,new_path):
        # Create PDF Parser
        parser = PDFParser(dataIo)
        # Create PDFDocument
        document = PDFDocument(parser)
        # Is it okay for extraction?
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            # Create PDF Manager
            resmag =PDFResourceManager()
            # Setting parameters
            laparams=LAParams()
            # Createing PDF device
            # device=PDFDevice(resmag )
            device=PDFPageAggregator(resmag ,laparams=laparams)
            # Create PDF explainer
            interpreter=PDFPageInterpreter(resmag ,device)
            # For each page
            for page in PDFPage.create_pages(document):
                interpreter.process_page(page)
                # accept this page's LTP object
                layout=device.get_result()
                for y in layout:
                    try:
                        if(isinstance(y,LTTextBoxHorizontal)):
                            with open('%s'%(new_path),'a',encoding="utf-8") as f:
                                f.write(y.get_text()+'\n')
                                #print("Success！")
                    except:
                        print("Failed")
    def rdflib_author_name(self,filename,index,totalList):
        #paper_uri=URIRef("http://www.scoaring.com/ontology#P1296596")
        mid_uri = URIRef("http://www.scoaring.com/ontology#")
        prefix = Namespace(mid_uri)
        with open(filename, "r",encoding='latin-1') as f:
            g = Graph()
            result = g.parse(f, format="turtle")
            print(len(g))
            paper=prefix['Paper']
            urlPDF=prefix['urlPDF']
            isContributionToPaper=prefix['isContributionToPaper']
            isContributionByAuthor=prefix['isContributionByAuthor']
            name=URIRef("http://xmlns.com/foaf/0.1/name")
            papers=prefix["makgAuthorPaperCount"]
            citations=prefix["makgAuthorCitationCount"]
            l=0
            #paper_list=[]
            author_list=[]
            contri_list=[]
            name_list=[]
            author_paper_number_list=[]
            author_citation_number_list=[]
            
            for s,p,o in g.triples( (paper_uri, RDF.type, paper) ):
                l+=1
                paper_id=s
                print(paper_id)
                #print(s)
                for s2,p2,o2 in g.triples((paper_id, urlPDF, None) ):
                    print(s2,p2,o2)
                #break
                for s3,p3,o3 in g.triples((None, isContributionToPaper, paper_id) ):
                    contri_list.append(s3)
                    #print(contri_list)
                    #print(s3,p3,o3)
                break  
            for contri in contri_list:
                for s4,p4,o4 in g.triples( (contri, isContributionByAuthor, None) ):
                    author_list.append(o4)
            print(author_list)
            for author in author_list:
                for s5,p5,o5 in g.triples( (author, name, None) ):
                    name_list.append(str(o5))
                for s6,p6,o6 in g.triples( (author, papers, None) ):
                    author_paper_number_list.append(int(o6))
                for s7,p7,o7 in g.triples( (author, citations, None) ):
                    author_citation_number_list.append(int(o7))
            print(name_list)
            print(author_paper_number_list)
            print(author_citation_number_list)
#             print()
#             print(l)
    
    def main(self,index):
        '''
        Main function of the class, generate related datastructure for further usage.
        Args:
            index: int as query_result's index
           
        Returns:
            resourceId: string represents each paper's unique ID, e.g. P12345
            authorList: A list containing strings of all the names of authors
            authorPaper: A dictionary containing integers of all the paper count of authors
            authorCitation: : A dictionary containing integers of all the citation counts of authors
            authorID: A dictionary containing strings of all the strings ID of authors. e.g. A12345
        '''
        query_result=self.connection()
#         for index in range(len(NUM)):
        resourceId,url=self.pre_processing(index,query_result)
        try:
            my_file=("txt/%s.txt")%resourceId
            my_file=Path(my_file)
            my_abs_path = my_file.resolve()
        except IOError:
            print("File is not accessible.Download from the url")
            html = urllib.request.urlopen(urllib.request.Request(url)).read()
            dataIo = BytesIO(html)
            self.onlinePdfToTxt(dataIo,'txt/'+str(resourceId)+'.txt')
        authorList,authorPaper,authorCitation,authorID=self.query(index,resourceId)
        return resourceId,authorList,authorPaper,authorCitation,authorID

# processing=MySQL_Processing()
# processing.rdflib_author_name("full_graph.ttl",'')
# processing.NUM=5
# resourceId,authorList,authorPaper,authorCitation,authorID=processing.main(4)
# print(authorList)

In [11]:
from io import StringIO
from io import BytesIO
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import *
from pdfminer.converter import PDFPageAggregator
from urllib.request import Request
from urllib.request import urlopen
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef,RDFS
from rdflib.namespace import DC, FOAF
from owlrl import DeductiveClosure, RDFS_Semantics
from pathlib import Path
import urllib
import pdfminer
import copy
import os.path
import pprint
import json

#input basic info about database
class RDF_Processing():
    '''
    This class is used to handle data input in RDF format
    '''
    def __init__(self,filename):
        self.prefix = Namespace(URIRef("http://www.scoaring.com/ontology#"))
        self.f=open(filename, "r",encoding='latin-1') 
        self.g = Graph()
        result = self.g.parse(self.f, format="turtle")
        
    def paper_url(self,filename,total_list,index):
        urlPDF=self.prefix['urlPDF']
        paper_id=total_list[index]
        for s,p,url in self.g.triples( (paper_id, urlPDF, None) ):
            if url[-3:]!='pdf':
                url=url+".pdf"
            return url
            
    def onlinePdfToTxt(self,dataIo,new_path):
        # Create PDF Parser
        parser = PDFParser(dataIo)
        # Create PDFDocument
        document = PDFDocument(parser)
        # Is it okay for extraction?
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            # Create PDF Manager
            resmag =PDFResourceManager()
            # Setting parameters
            laparams=LAParams()
            # Createing PDF device
            # device=PDFDevice(resmag )
            device=PDFPageAggregator(resmag ,laparams=laparams)
            # Create PDF explainer
            interpreter=PDFPageInterpreter(resmag ,device)
            # For each page
            for page in PDFPage.create_pages(document):
                interpreter.process_page(page)
                # accept this page's LTP object
                layout=device.get_result()
                for y in layout:
                    try:
                        if(isinstance(y,LTTextBoxHorizontal)):
                            with open('%s'%(new_path),'a',encoding="utf-8") as f:
                                f.write(y.get_text()+'\n')
                                #print("Success！")
                    except:
                        print("Failed")
            
    def paper_list(self,filename):
        total_list=[]
        paper=self.prefix['Paper']
        urlPDF=self.prefix['urlPDF']
        for s,p,o in self.g.triples( (None, RDF.type, paper) ):
            paper_id=s
            total_list.append(paper_id)
        return total_list

    def rdf_extraction(self,filename,total_list,index):
        '''
        This class is used to extract relevant paper's info from RDF triples store
        Args:
            filename: file containing rdf triple
            total_list: list contraining each paper's id in total.
            index:current paper's index in list
        '''
        paper=self.prefix['Paper']
        isContributionToPaper=self.prefix['isContributionToPaper']
        isContributionByAuthor=self.prefix['isContributionByAuthor']
        name=URIRef("http://xmlns.com/foaf/0.1/name")
        papers=self.prefix["makgAuthorPaperCount"]
        citations=self.prefix["makgAuthorCitationCount"]
        author_list=[]
        contri_list=[]
        author_id_list=[]
        author_paper_number_dict={}
        author_citation_number_dict={}
        s1=str(total_list[index]).find("#")
        paper_id=str(total_list[index])[s1+1:]

        name_dict={}
        for s3,p3,o3 in self.g.triples((None, isContributionToPaper, (total_list[index])) ):
            contri_list.append(s3)
        for contri in contri_list:
            for s4,p4,o4 in self.g.triples( (contri, isContributionByAuthor, None) ):
                author_list.append(o4)
        for author in author_list:
            s=str(author).find("#")
            aid=author[s+1:]
            author_id_list.append(author[s+1:])
            for s5,p5,o5 in self.g.triples( (author, name, None) ):
                name_dict[aid]=(str(o5))
            for s6,p6,o6 in self.g.triples( (author, papers, None) ):
                author_paper_number_dict[aid]=(int(o6))
            for s7,p7,o7 in self.g.triples( (author, citations, None) ):
                author_citation_number_dict[aid]=(int(o7))
        return paper_id,author_id_list,author_paper_number_dict,author_citation_number_dict,name_dict
    
    def main(self,filename,total_list,index):
        paper_id,author_id_list,author_paper_number_dict,author_citation_number_dict,name_dict=self.rdf_extraction(filename,total_list,index)
        url=self.paper_url(filename,total_list,index)
        #print(paper_id)
#         try:
        my_file=("txt/%s.txt")%paper_id
        #my_file=Path(my_file)
        if os.path.exists(my_file):
            #my_abs_path = my_file.resolve()
            print(("%s exist")%paper_id)
        else:
#         except IOError:
            print("File is not accessible.Download from the url")
            html = urllib.request.urlopen(urllib.request.Request(url)).read()
            dataIo = BytesIO(html)
            self.onlinePdfToTxt(dataIo,'txt/'+str(paper_id)+'.txt')
        return paper_id,author_id_list,author_paper_number_dict,author_citation_number_dict,name_dict
            
# rdf=RDF_Processing()
# rdf.main("before.ttl",total_list,3)

After excution we obtained a 2-dimensional matrix with each row represents a paper. Next, we constract a url based on arXiv id.

In [2]:
import spacy
import re
import copy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from Levenshtein import *
#nlp = en_core_web_sm.load()
class AuthorInfoExtract():
    '''
    This class is used to extract special author with annotations from raw text.
    The result is organized as a list containing dictionaries
    '''
    STARS=["*",'∗']
    CROSS=["†","‡"]
    def detectCapitalUse(self, word: str) -> bool:
        '''
        Function used to judge author name's capital letters 
        '''
        is_upper = [c.isupper() for c in word]          # Upper 
        is_lower = [c.islower() for c in word]          # Lower             

        if any(is_upper):                               # Upper and lower
            return is_upper[0] and all(is_lower[1:])    # Only first is upper 
        
    def locateContext(self,paper_id):
        '''
        Locate where the authors' information at.
        Args:
            resourceId: string represents each paper's unique ID, e.g. P12345
        Returns:
            output: string containing the related information
        '''
        file = open('txt/'+str(paper_id)+'.txt',encoding="utf-8")
        strings=file.read()
        symbolList=["Abstract",'Abstract.','ABSTRACT','ABSTRACT.'] # different version
        for symbol in symbolList:
            if strings.find(symbol):
                output=strings[0:strings.find(symbol)]
                break
            else:
                print(symbol+" not found!")
                output=strings[0:3000]
        file.close()
        output=output.replace('\n', ' . ').replace('\r', ' ')
        output=output.split(" ")
        return (output)
    
    def acknowledgementJudgement(self,paper_id,name_dict,corre):
        '''
        Judge the acknowlegement section whether containing corresponding author's information
        Args:
            resourceId: string represents each paper's unique ID, e.g. P12345
            authorList: A list containing strings of all the names of authors
            corres: A List containing the corresponding author's information e.g. [0,0,1,0]
        Returns:
            corres: A List containing the corresponding author's information after processing  
            e.g. [0,0,1,1]
        '''
        file = open('txt/'+str(paper_id)+'.txt',encoding="utf-8")
        strings=file.read()
        symbolList=["Acknowledgement",'ACKNOWLEDGEMENT','Acknowledgements','ACKNOWLEDGEMENTS','ACK']
        for symbol in symbolList:
            #if name in strings:
            if strings.find(symbol):
                output=strings[strings.find(symbol):strings.find(symbol)+1000]
                break
            else:
                print(symbol+" not found!")
        file.close()
        output=output.replace('\n', ' . ').replace('\r', ' ')
        output=output.split(" ")
        output=" ".join(output)
        if output.find("corresponding"):
            pos= output.find("corresponding")
            print(pos)
            #judge whether there is any name in the author list in the context
            for k,v in name_dict.items():
                if v in strings[pos-30:pos+30]:
                    corre.append(author)
        return corre
    def author_contribution(self,paper_id):
        file = open('txt/'+str(paper_id)+'.txt',encoding="utf-8")
        strings=file.read()
        symbolList=["Author Contributions","Author Contribution"]
        for symbol in symbolList:
            res=bool(re.search(symbol, strings, re.IGNORECASE))
            if res==True:
                return True
            else:
                continue
        return False
            
    def symbolJudgement(self,paper_id):
        '''
        Judge symbol represent what in the context.
        Args:
            resourceId: string represents each paper's unique ID, e.g. P12345
        Returns:
            num: integer, represent different situations
            0:default
            1:star represent equall 
            2:star represent correspinding 
            3:order ranged by alphabetical order
        '''
        file = open('txt/'+str(paper_id)+'.txt',encoding="utf-8")
        strings=file.read()
        strings=strings[:3000]
        #print(strings)
        if strings.find("equally"):
            pos= strings.find("equally")
            #print(pos)
            for star in self.STARS:
                if star in strings[pos-50:pos+50]:
                    return 1 #star represent equall 
        if strings.find("Corresponding"):
            pos= strings.find("Corresponding")
            for star in self.STARS:
                if star in strings[pos-50:pos+50]:
                    return 2 #star represent correspinding 
        if "Alphabetical" in strings:
            return 3
        return 0
    
    def matchAuthor(self,roleList,authorList):
        '''
        Judge whether author names recognized in the context are the same with author names from mysql
        Args:
            roleList: A list containing author names recognized from the context
            authorList: A list containing strings of all the names of authors
        Returns:
            author_role: A list containing names represent for speical authors
        '''
        #authorList=list(name_dict.values())
        author_role=[]
        for role in roleList:
            highest=0
            index=0
            for i in range(len(authorList)):
                if jaro(role, authorList[i])>highest:
                    index=i
                    highest=jaro(role, authorList[i])
            #print(highest,authorList[index])
            if highest>0.7:
                author_role.append(authorList[index])
        return author_role
    
    def specialAuthors(self,output,name_dict,resourceId):
        '''
        Determine special authors
        Args:
            output: string containing the related information
            authorList: A list containing strings of all the names of authors
            resourceId: string represents each paper's unique ID, e.g. P12345
        Returns:
            star:A list of name represent for first authors
            corre:A list of names represent for corresponding author
        '''
        authorList=list(name_dict.values())
        starList=[]
        crossList=[]
        for i in range(len(output)):
            for star in self.STARS:
                if star in output[i]:
                    starList.append(" ".join(output[i-2:i+1]))
            for cross in self.CROSS:
                if cross in output[i]:
                    crossList.append(" ".join(output[i-2:i+1]))
        judge=self.symbolJudgement(resourceId)
        #print(judge)
        if judge==0 or judge==1:
            star=self.matchAuthor(starList,authorList)
            corre=self.matchAuthor(crossList,authorList) 
        elif judge==2:
            star=self.matchAuthor(starList,authorList)
            corre=copy.copy(star)
            star=[]
        elif judge==3:
            star=[]
            corre=[]
        #print(star)
        return star,corre

    def sequenceExtratcor(self,output,name_dict,parameter=0):
        '''
        Extract the sequence from the author list
        Args:
            output: string containing the related information
            authorList: A list containing strings of all the names of authors
            parameter: 0 represents return the result while 1 represents return original authorList
        Returns:
            author_sequence: A dictionary containing the order of authors, e.g. {AAA:1}
        '''
        authorList=list(name_dict.values())
        #print(type(authorList))
        author_sequence={}
        author_sequence_list=[]
        authorListCopy=copy.copy(authorList)
        num=1
        for i in range(len(output)):
            for name in authorListCopy:
                if output[i] in name and len(output[i])>=2 and self.detectCapitalUse(output[i]): #Detect capital
                    author_sequence[name]=num
                    author_sequence_list.append(name)
                    num+=1
                    authorListCopy.remove(name)
                    break
        if len(author_sequence_list)==len(authorList):
            authorList=author_sequence_list
        if parameter==0:
            return author_sequence
        else:
            return authorList
    
    def main(self,paper_id,author_id_list,author_paper_number_dict,author_citation_number_dict,name_dict):
        
        '''
        main function of class AuthorInfoExtract
        return:
            A dataframe containing all the information extracted
        '''
        extractor=AuthorInfoExtract()
        
        output=extractor.locateContext(paper_id)
        #authorList=name_dict.values()
        #print(output)
        author_sequence=extractor.sequenceExtratcor(output,name_dict)
        print(author_sequence)
        star,corre=extractor.specialAuthors(output,name_dict,paper_id)
        output2=extractor.acknowledgementJudgement(paper_id,name_dict,corre)
        print(output2)
        #print(star)
#         try:
        df=pd.DataFrame.from_dict(author_sequence,orient='index',columns=['sequence'])
        df=df.reset_index().rename(columns={'index':'name'})
        df['authorID']=0
        for index, row in df.iterrows():
            for k,v in name_dict.items():
                #print(row["name"],author,index)
                if row["name"]==v:
                    df.loc[index,"authorID"] = k
        df['star']=0
        for index, row in df.iterrows():
            for author in star:
                if row["name"]==author:
                    df.loc[index,"star"] = 1
        df['corre']=0
        for index, row in df.iterrows():
            for author in corre:
                #print(row["name"],author,index)
                if row["name"]==author:
                    df.loc[index,"corre"] = 1
        df['paperCount']=0
        for index, row in df.iterrows():
            for k,v in author_paper_number_dict.items():
                #print(row["name"],author,index)
                if row["authorID"]==k:
                    df.loc[index,"paperCount"] = v

        df['citationCount']=0
        for index, row in df.iterrows():
            for k,v in author_citation_number_dict.items():
                #print(row["name"],author,index)
                if row["authorID"]==k:
                    df.loc[index,"citationCount"] = v

#         except:
#             print("error occured!")
#             df=pd.DataFrame(columns=['name', 'sequence', 'star', 'corre'])
        return df
    
# extractor=AuthorInfoExtract()
# annotation=extractor.main()
# annotation

In [3]:
def lookup_id(author,df):
    '''
    Given dataframe and name return the author ID for triple
    Args:
        author:string of author name
        df:A dataframe generated by the class AuthorInfoExtract 
    Returns:
        authorID: A string represent unique Id of author. e.g. A12345
    '''
    author_id=""
    for index, row in df.iterrows():
        if author==row["name"]:
            author_id=(df.at[index,"authorID"])
            break
    return author_id

def lookup_order(author,df):
    '''
    Given dataframe and name return the author ID for triple
    Args:
        author:string of author name
        df:A dataframe generated by the class AuthorInfoExtract 
    Returns:
        authorID: A string represent unique Id of author. e.g. A12345
    '''
    author_id=""
    for index, row in df.iterrows():
        if author==row["name"]:
            author_order=(df.at[index,"sequence"])
            break
    return int(author_order)

def roleGenerator(df,resourceId):
    '''
    Given dataframe and resourceId to determine the role of each author
    Args:
        df:A dataframe generated by the class AuthorInfoExtract 
        resourceId: string represents each paper's unique ID, e.g. P12345 Used for naming the contribution
    Returns:
        result, A list containing contribution_id,resourceId,author_id,role,author_order for further triple generation
    '''
    ROLES=['Writing_Draft','Writing_Review_Editing','Supervision','Resource','Software']
    LEVELS=[10,50,100,200,1000,3000,5000,10000]
    df_role = pd.DataFrame(0.0, index=df.name, columns=ROLES)
    #sieve1:Sequence
    for index in range(len(df)):
        df_role.loc[df.name[index],"Writing_Draft"] =1/df.sequence[index]
        df_role.loc[df.name[index],"Writing_Review_Editing"] =(df.sequence[index]/len(df.sequence))
    #Last author is likely to be supervisor
    
    #sieve2:Annotation
    for index in range(len(df)):
        if df.star[index]==1:
            df_role.loc[df.name[index],"Writing_Draft"] +=1
        if (len(df.star[df.star>0]))<=2:
            if df.corre[index]==1:
                df_role.loc[df.name[index],"Writing_Review_Editing"] +=1
                df_role.loc[df.name[index],"Supervision"] +=0.5
            
    #sieve3:Citation and Paper
    for index in range(len(df)):
        df_role.Supervision[index]=df.citationCount[index]*0.2/sum(df.citationCount)+df.paperCount[index]*0.8/sum(df.paperCount)
    for index in range(len(df)):
        num=0.0
        for level in LEVELS:
            if df.citationCount[index]>level:
                num+=0.125
            if df.paperCount[index]>level:
                num+=0.125
        df_role.Resource[index]=num
        df_role.Writing_Review_Editing[index]+=num/5
    print(df_role) 
    author_contribution={}
    paper_contribution={}
    contribution_role={}
    result=[]
    result_1=[]
    result_2=[]
    result_3=[]
#     try:
    if len(df_role.Supervision)>0:
        supervisionMax=max(df_role.Supervision)
    for index in range(len(df_role)):
        try:
            author_id=lookup_id(df.name[index],df)
            author_order=lookup_order(df.name[index],df)
            print(author_id)
            flag=False
            if df_role.loc[df.name[index],"Writing_Draft"]>=1:
                contribution_id=resourceId+"_"+author_id+"_writing_Draft"
                role="writing_Draft"
                result.append([contribution_id,resourceId,author_id,role,author_order])
                flag=True
            if df_role.loc[df.name[index],"Writing_Review_Editing"]>=1:
                contribution_id=resourceId+"_"+author_id+"_writing_review_editing"
                role="writing_review_editing"
                result.append([contribution_id,resourceId,author_id,role,author_order])
                flag=True
            if df_role.loc[df.name[index],"Resource"]>=1:
                contribution_id=resourceId+"_"+author_id+"_resource"
                role="resource"
                result.append([contribution_id,resourceId,author_id,role,author_order])
                flag=True
            if df_role.loc[df.name[index],"Supervision"]==supervisionMax:
                contribution_id=resourceId+"_"+author_id+"_supervision"
                role="supervision"
                result.append([contribution_id,resourceId,author_id,role,author_order])
                flag=True
            if flag==False:
                contribution_id=resourceId+"_"+author_id+"_default"
                role="default"
                result.append([contribution_id,resourceId,author_id,role,author_order])
        except:
            print("fail to generate role")
    return result

#roleGenerator(annotation)


In [27]:
from Levenshtein import *
import re
class EmailInfoExtract():
    '''
    This class is used to extract email address and match with the authors
    The result is orgnized as a list containing dictionaries
    '''
    def __init__(self):
        self.name_email={}
        self.result=[]
    def get_alpha_str(self,s):
        result = ''.join([x for x in s if x.isalpha()])
        return result

    def email_extraction(self,file_name):
        '''
        This class handle raw text to extract email addresses.
        '''
        file = open('txt/'+str(file_name)+'.txt',encoding="utf-8")
        strings=file.read()
        #First page's txt
        strings=strings[:3000]
        matches = []
        matchesGroup=[]
        #1. singular email address
        emailRegex = re.compile(r'''(
            [a-zA-Z0-9._%+-]+      # username
            @+                     # @ symbol

           [a-zA-Z0-9_-]+
           (\.[a-zA-Z0-9_-]+)+
            )''', re.VERBOSE)
        # using RE to match all the patterns in the txt
        for groups in emailRegex.findall(strings):
            matches.append(groups[0])

        list2 = matches
        list_nums = len(list2)
        emailList=[]
        for line in range(list_nums):
            emailList.append(list2[line])
        #2. grouped email list {}
        emailRegex2 = re.compile(r'''(
        \{(.+?)\}+
        @                      # @ symbol

        [a-zA-Z0-9.-]+        # domain name

        (\.[a-zA-Z]{1,4}){1,2} # dot-something
        )''', re.VERBOSE)
        for groups in emailRegex2.findall(strings):
            #print(groups)
            matchesGroup.append(groups[0])
        list3 = matchesGroup
        if len(matchesGroup)>0:
            for gp in list3:
                stringGroup = gp
                index=stringGroup.find("@", 0)
                prefix=stringGroup[:index]
                suffix=stringGroup[index:]
                tempNameList=prefix[1:-1].split(",");
                for name in tempNameList:
                    emailList.append(name+suffix)
                #print(emailList)
        #3. convert to sequenced list if possible 
        sequencedList=[]
        temp_dict={}
        for mail in emailList:
            index=mail.find("@",0)
            prefix=mail[:index]
            email_index=strings.find(prefix)
            temp_dict[mail]=email_index
        sort_idct=sorted(temp_dict.items(),key = lambda x:x[1],reverse = False)
        for k,v in sort_idct:
            sequencedList.append(k)
        #print((sequencedList))

        if len(sequencedList)==len(emailList):
            return sequencedList
        else:
            return emailList
        
    def lookup_id(self,author,df):
        '''
        This class return author's ID by looking up the dataframe
        '''
        author_id=""
    #     for index, row in df.iterrows():
        for index, row in df.iterrows():
            #print(index)
            #print(k1,v2)
            if author==row["name"]:
                author_id=(df.at[index,"authorID"])
                break
        return (author_id)

    def order_matching(self,nameList, emailList):   
        '''
        If the email address number is exactly same as the number of authors, we made 1 to 1 matching directly.
        args:
            nameList: Ordered list containing the sequence of authors.
            emailList: Extracted email List from the raw txt.
        '''
        print("ORDER MATCHING")
        for i in range(len(nameList)):
            name=nameList[i]
            email=emailList[i]
            self.name_email[name]=email
        return self.name_email

    #2.No macthes, need to group based on experience and naive approach
    def naive_matching(self,nameList, emailList):
        '''
        If the length are not matched, we try to find the most likely pair by jaro algorithm.
        args:
            nameList: Ordered list containing the sequence of authors.
            emailList: Extracted email List from the raw txt.
        '''
        print("NAIVE MATCHING")
        for name in nameList:
            highest=0
            index=0
            for i in range(len(emailList)):
                #using re to extract the first part of email
                pat = re.compile(''+'(.*?)'+'@', re.S)
                email = pat.findall(emailList[i])
                email= self.get_alpha_str(email)
                #using jaro to calculate the similarity between two strings
                if jaro(name, str(email))>highest:
                    index=i
                    highest=jaro(name, str(email))
            #set pair with the highest score
                    print(name,highest,emailList[i])
            #self.name_email[name]=emailList[index]
            #If very sure, remove it from the list to reduce the uncertainty for other pairs
            if highest>=0.5:
                self.name_email[name]=emailList[index]
                emailList.remove(emailList[index])
        return self.name_email
    
    def main(self,paper_id,name_dict,df):
        extractor=AuthorInfoExtract()
        output=extractor.locateContext(paper_id)
        name_list=extractor.sequenceExtratcor(output,name_dict,1)
        email_list=self.email_extraction(paper_id)
        #print(emailList)
        if len(email_list)==0:
            return
        name_email={}
        print(email_list)
        #1.If sequenced list and length matches
        if len(name_list)==len(email_list):
            self.name_email=self.order_matching(name_list, email_list)
        else:
            self.name_email=self.naive_matching(name_list, email_list)
        
        for k,v in self.name_email.items():
            author_id=self.lookup_id(k,df)
            #print(author_id)
            temp={}
            temp[author_id]=v
            self.result.append(temp)
        return self.result
        #return self.name_email
    
# emailExtractor=EmailInfoExtract()
# name_email=emailExtractor.main()

# name_email

In [5]:
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef,RDFS
from rdflib.namespace import DC, FOAF
from owlrl import DeductiveClosure, RDFS_Semantics
import pprint
import json
def create_triple_email(totalList):
    '''
    Create triple and serialize it from the list we generated by the end.
    args:
        totalList: A list contraining sublists, which have dictionaries of author_id:email 
        e.g. [[{A123:aaa@email.com},{...},[{A345:bbb@email.com}]]
    return:
        serialized file, saved as txt
    '''
    g = Graph()
    email_address=URIRef("http://www.scoaring.com/ontology#emailAddress")
    for lst in totalList:
        if lst!=None:
            for dic in lst:
                for author_id,v1 in dic.items():  
                    author = URIRef("http://www.scoaring.com/ontology#%s")%author_id
                    g.add((author,email_address, Literal(v1)))
    print(len(g))
    print(g.serialize(format='turtle').decode())
    g.serialize(destination='email_final.ttl', format='turtle')
    
def modify_triple_role(ttl,totalList):
    with open(ttl, "r",encoding='latin-1') as f:
        g = Graph()
        result = g.parse(f, format="turtle")
        #g.parse(ttl, format='turtle').decode('latin-1')
        #print(len(g))
        for subList in totalList:
            for contribution_id,paper_id,author_id,role,order in subList:
                contribution_unspecified = URIRef("http://www.scoaring.com/ontology#%s_%s_unspecified")%(paper_id,author_id)
                #1. Remove original unspecified triple from raw data.
                if (contribution_unspecified, None, None ) in g:
                    print("##############original#####################")
                    for s,p,o in g.triples((contribution_unspecified, None,None)):
                        #print(s,p,o)
                        g.remove((contribution_unspecified, None, None))
                #2. Create triples from generated results
                numInList=URIRef("http://www.scoaring.com/ontology#orderInListOfAuthors")#placeInListOfAuthors
                isRole= URIRef("http://www.scoaring.com/ontology#role")#role
                contribution = URIRef("http://www.scoaring.com/ontology#%s")%contribution_id
                paper=URIRef("http://www.scoaring.com/ontology#%s")%paper_id
                author=URIRef("http://www.scoaring.com/ontology#%s")%author_id
                typeClass=URIRef("http://www.scoaring.com/ontology#Contribution")
                isContributionByAuthor= URIRef("http://www.scoaring.com/ontology#isContributionByAuthor")#refersTo
                isContributionToPaper= URIRef("http://www.scoaring.com/ontology#isContributionToPaper")#refersTo
                contribution_role=Literal(role)
                namedIndividual = URIRef('http://www.w3.org/2002/07/owl#NamedIndividual')
                g.add((contribution,isRole,contribution_role))
                g.add((contribution, RDF.type, typeClass))
                g.add((contribution, RDF.type, namedIndividual))
                g.add((contribution,isContributionByAuthor, author))
                g.add((contribution,isContributionToPaper, paper))
                g.add((contribution,numInList,Literal(order)))

    f.close()
    g.serialize(destination=('full_graph_final.ttl'), format='turtle')

In [ ]:
import json
def main():
    FILENAME="ttl/before.ttl"# file to process,(all contributions are defined as unspecified)
    #processing=Processing()
    email=[]
    role_result=[]
    count=1
    ce=1
    rdf=RDF_Processing(FILENAME)
    total_list=rdf.paper_list(FILENAME)#a list contraining all paper ID
    for i in range(len(total_list)):#For each paper
        #each of the paper
        try:
            print("##################")
            print(count)
            paper_id,author_id_list,author_paper_number_dict,author_citation_number_dict,name_dict=rdf.main(FILENAME,total_list,i)
    #             print(authorList)
            extractor=AuthorInfoExtract()
            annotation=extractor.main(paper_id,author_id_list,author_paper_number_dict,author_citation_number_dict,name_dict)
            print(annotation)
            result=roleGenerator(annotation,paper_id)
            role_result.append(result)
            #print(role_result)
            emailExtractor=EmailInfoExtract()
            name_email=emailExtractor.main(paper_id,name_dict,annotation)
            #print(name_email)
            email.append(name_email)
            print(name_email)
            count+=1
        except:
            ce+=1
            print(ce)
            if ce>20:
                modify_triple_role(FILENAME,role_result)
                create_triple_email(email)
                print("finished")
                break
            continue
    print(ce)
    modify_triple_role(FILENAME,role_result)
    create_triple_email(email)
    print("finished")
    #modify_triple_role("Dataset v4 17-06-2020.ttl",role_result)
    #json_role = json.dumps(role_result)
#    create_triple_email(email)
#     with open("role.json", 'w') as f:
#         json.dump(json_role, f)
    #create_triple_email(email)
main()

##################
1
http://papers.nips.cc/paper/7363-connectionist-temporal-classification-with-maximum-entropy-regularization.pdf
http://papers.nips.cc/paper/7363-connectionist-temporal-classification-with-maximum-entropy-regularization.pdf
P1170488 exist
{'Hu Liu': 1, 'Sheng Jin': 2, 'Changshui Zhang': 3}
-1
[]
              name  sequence authorID  star  corre  paperCount  citationCount
0           Hu Liu         1   A56003     0      0           1              0
1        Sheng Jin         2  A128993     0      0           1              0
2  Changshui Zhang         3   A23132     0      0         371           8802
                 Writing_Draft  Writing_Review_Editing  Supervision  Resource  \
name                                                                            
Hu Liu                1.000000                0.333333     0.002145     0.000   
Sheng Jin             0.500000                0.666667     0.002145     0.000   
Changshui Zhang       0.333333                1.

In [183]:
def contribution_paper_list():
    processing=Processing()
    extractor=AuthorInfoExtract()
    processing.NUM=2000#All papers
    email=[]
    author_contribution_paper=["1"]
    count=1
    ce=1
    for i in range(processing.NUM):
        try:
            print("##################")
            print(count)
            resourceId=processing.main(i)
            resourceId,authorList,authorPaper,authorCitation,authorID=processing.main(i)
            res=extractor.author_contribution(resourceId)
            print(res)
            if res ==True:
                author_contribution_paper.append(resourceId)
                print(resourceId)
            count+=1
        except:
            ce+=1
            print(ce)
            if ce>50:
                print(author_contribution_paper)
                f = open('contribution_paper_list.txt','w') #output.txt - 文件名称及格式 w - writing
                f.write(str(author_contribution_paper))
                f.close()
                print("finished")
                break
contribution_paper_list()

##################
1
http://ma-graph.org/entity/2618494520 https://github.com/gingsmith/fmtl
http://ma-graph.org/entity/2618494520 https://github.com/gingsmith/fmtl
2
##################
1
http://ma-graph.org/entity/2099749431 https://github.com/soheilb/multimodal_dictionary_learning
http://ma-graph.org/entity/2099749431 https://github.com/soheilb/multimodal_dictionary_learning
3
##################
1
http://ma-graph.org/entity/2803890652 https://github.com/MIC-DKFZ/TractSeg
http://ma-graph.org/entity/2803890652 https://github.com/MIC-DKFZ/TractSeg
4
##################
1
http://ma-graph.org/entity/2611292810 https://github.com/yhldhit/WMMD-Caffe
http://ma-graph.org/entity/2611292810 https://github.com/yhldhit/WMMD-Caffe
5
##################
1
http://ma-graph.org/entity/2770884134 https://github.com/ermongroup/InfoGAIL
http://ma-graph.org/entity/2770884134 https://github.com/ermongroup/InfoGAIL
6
##################
1
http://ma-graph.org/entity/2798702047 https://github.com/urvashik/lm-con

In [30]:
# g1 = Graph()
# g1.parse("role_sequential_final.ttl", format='turtle')
# print("g1 has {} triples".format(len(g1)))

# g2 = Graph()
# g2.parse("email_sequential.ttl", format='turtle')
# print("g2 has {} triples".format(len(g2)))

# graph = g1 + g2
# print("g1 + g2 has {} triples".format(len(graph)))
# graph.serialize(destination=('Dataset_v4.ttl'), format='turtle')
g1 = Graph()
g1.parse("full_graph_final.ttl", format='turtle')
print("g1 has {} triples".format(len(g1)))

g3=Graph()
g3.parse("email_final.ttl", format='turtle')
print("g3 has {} triples".format(len(g3)))
graph = g1 + g3
print("g1 + g2 has {} triples".format(len(graph)))
graph.serialize(destination=('v5_full_graph.ttl'), format='turtle')

g1 has 156508 triples
g3 has 3147 triples
g1 + g2 has 159655 triples
